In [ ]:
#preprocessing part 3

#features being extracted:
'''
To diagnose CAD, a doctor looks for changes to the ECG waves or segments. Changes that could indicate CAD or a heart attack include:

ST segment depression
ST segment elevation
flattening of the T waves
inverted T waves
Q waves
'''

In [1]:
import numpy as np
import neurokit2 as nk
import pandas as pd
import os
from tqdm import tqdm
from scipy.signal import medfilt
import warnings

In [2]:
warnings.filterwarnings('ignore')


In [3]:
# --- Configuration ---
FS = 500
LEAD_COUNT = 12
SAMPLES = 2500

# Standard 12-lead ECG ordering: I, II, III, aVR, aVL, aVF, V1, V2, V3, V4, V5, V6
LEAD_II = 1   # For rhythm analysis
LEAD_V1 = 6   # For R/S ratio
LEAD_V2 = 7   # For ST/T analysis
LEAD_V5 = 10  # For ST/T analysis and baseline

CSV_PATH = 'baseline_filtered_ecg_patients_2.csv'
FEATURE_SAVE_DIR = './extracted_clinical_features'
OUTPUT_CSV_PATH = 'metadata_with_features.csv'

In [4]:
# --- Feature Extraction Function ---
def extract_cad_features(signal, fs=FS, debug=False):
    """
    Extracts 8 clinically relevant features for CAD severity prediction.
    
    Returns:
        np.ndarray of 8 features or None if extraction fails
    """
    if signal.shape[0] != LEAD_COUNT or signal.shape[1] != SAMPLES:
        if debug:
            print(f"Invalid shape: {signal.shape}")
        return None
    
    # Use Lead II for rhythm analysis
    lead_ii = np.ascontiguousarray(signal[LEAD_II, :], dtype=np.float64)
    
    features = np.zeros(8)
    
    try:
        # Step 1: R-Peak Detection
        r_peaks_info = nk.ecg_findpeaks(lead_ii, sampling_rate=fs, method='neurokit')
        r_peaks = np.array(r_peaks_info['ECG_R_Peaks'])
        
        # Require at least 3 R-peaks for reliable feature extraction
        if len(r_peaks) < 3:
            if debug:
                print(f"Too few R-peaks detected: {len(r_peaks)}")
            return None
        
        # Step 2: Delineation with better error handling
        try:
            _, waves_info = nk.ecg_delineate(
                lead_ii, 
                rpeaks=r_peaks, 
                sampling_rate=fs, 
                method="dwt"
            )
        except Exception as e:
            if debug:
                print(f"Delineation failed: {e}")
            return None
        
        # Step 3: Find the central beat
        center_idx = np.argmin(np.abs(r_peaks - SAMPLES // 2))
        r_peak_idx = r_peaks[center_idx]
        
        # Step 4: Extract wave boundaries with validation
        def get_wave_index(wave_dict, key, beat_idx):
            """Safely extract wave index with None checking"""
            if key not in wave_dict:
                return None
            val = wave_dict[key][beat_idx]
            if val is None or np.isnan(val):
                return None
            return int(val)
        
        q_onset = get_wave_index(waves_info, 'ECG_Q_Peaks', center_idx)
        s_offset = get_wave_index(waves_info, 'ECG_S_Peaks', center_idx)
        t_peak = get_wave_index(waves_info, 'ECG_T_Peaks', center_idx)
        
        # If any critical point is missing, try alternative detection
        if q_onset is None:
            # Q onset approximation: 40ms before R peak
            q_onset = max(0, r_peak_idx - int(0.04 * fs))
        
        if s_offset is None:
            # S offset approximation: 40ms after R peak
            s_offset = min(SAMPLES - 1, r_peak_idx + int(0.04 * fs))
        
        if t_peak is None:
            # T peak approximation: 200ms after R peak
            t_peak_approx = min(SAMPLES - 1, r_peak_idx + int(0.2 * fs))
            # Find actual peak in window
            search_start = r_peak_idx + int(0.1 * fs)
            search_end = min(SAMPLES, r_peak_idx + int(0.35 * fs))
            if search_end > search_start:
                t_peak = search_start + np.argmax(lead_ii[search_start:search_end])
            else:
                t_peak = t_peak_approx
        
        # Validate indices
        if not (0 <= q_onset < r_peak_idx < s_offset < SAMPLES):
            if debug:
                print(f"Invalid wave sequence: Q={q_onset}, R={r_peak_idx}, S={s_offset}")
            return None
        
        if not (0 <= t_peak < SAMPLES):
            if debug:
                print(f"Invalid T peak: {t_peak}")
            return None
        
        # Step 5: Calculate baselines for each lead independently
        baseline_v2 = medfilt(signal[LEAD_V2, :], kernel_size=11)
        baseline_v5 = medfilt(signal[LEAD_V5, :], kernel_size=11)
        baseline_ii = medfilt(signal[LEAD_II, :], kernel_size=11)
        
        # Step 6: Extract Features with robust error handling
        
        # Feature 0: Q Wave Depth (Lead II)
        q_region_start = max(0, q_onset)
        q_region_end = min(SAMPLES, r_peak_idx)
        if q_region_end > q_region_start:
            q_peak_idx = q_region_start + np.argmin(signal[LEAD_II, q_region_start:q_region_end])
            q_depth = baseline_ii[q_peak_idx] - signal[LEAD_II, q_peak_idx]
            features[0] = q_depth if not (np.isnan(q_depth) or np.isinf(q_depth)) else 0
        else:
            features[0] = 0
        
        # ST segment point: 60ms after S offset
        st_point_idx = min(SAMPLES - 1, s_offset + int(0.06 * fs))
        
        # Feature 1: ST Deviation in V2
        st_dev_v2 = signal[LEAD_V2, st_point_idx] - baseline_v2[st_point_idx]
        features[1] = st_dev_v2 if not (np.isnan(st_dev_v2) or np.isinf(st_dev_v2)) else 0
        
        # Feature 2: ST Deviation in V5
        st_dev_v5 = signal[LEAD_V5, st_point_idx] - baseline_v5[st_point_idx]
        features[2] = st_dev_v5 if not (np.isnan(st_dev_v5) or np.isinf(st_dev_v5)) else 0
        
        # Feature 3: T Wave Magnitude in V2
        t_mag_v2 = signal[LEAD_V2, t_peak] - baseline_v2[t_peak]
        features[3] = t_mag_v2 if not (np.isnan(t_mag_v2) or np.isinf(t_mag_v2)) else 0
        
        # Feature 4: T Wave Magnitude in V5
        t_mag_v5 = signal[LEAD_V5, t_peak] - baseline_v5[t_peak]
        features[4] = t_mag_v5 if not (np.isnan(t_mag_v5) or np.isinf(t_mag_v5)) else 0
        
        # Feature 5: QRS Duration (seconds)
        qrs_duration = (s_offset - q_onset) / fs
        features[5] = qrs_duration if 0.04 <= qrs_duration <= 0.20 else 0.10  # Physiologically valid range
        
        # Feature 6: R/S Ratio in V1
        qrs_region_v1 = signal[LEAD_V1, q_onset:s_offset]
        if len(qrs_region_v1) > 0:
            r_mag_v1 = np.max(qrs_region_v1) - np.median(qrs_region_v1)  # Relative to baseline
            s_mag_v1 = np.median(qrs_region_v1) - np.min(qrs_region_v1)  # Relative to baseline
            
            # Both should be positive; if not, use absolute values
            r_mag_v1 = abs(r_mag_v1)
            s_mag_v1 = abs(s_mag_v1)
            
            if s_mag_v1 > 0.01:  # Threshold to avoid division by near-zero
                rs_ratio = r_mag_v1 / s_mag_v1
                features[6] = rs_ratio if not (np.isnan(rs_ratio) or np.isinf(rs_ratio)) else 1.0
            else:
                features[6] = 1.0  # Default when S wave is absent/minimal
        else:
            features[6] = 1.0
        
        # Feature 7: Lead II RMS
        rms_ii = np.sqrt(np.mean(lead_ii**2))
        features[7] = rms_ii if not (np.isnan(rms_ii) or np.isinf(rms_ii)) else 0
        
        # Sanity check: all features should be finite and within reasonable ranges
        if np.any(~np.isfinite(features)):
            if debug:
                print(f"Non-finite feature values detected: {features}")
            return None
        
        # Validate physiological ranges (relaxed for pathological cases)
        checks = [
            (abs(features[0]) < 2.0, "Q wave depth"),      # Q depth typically < 2mV
            (abs(features[1]) < 2.0, "ST dev V2"),          # ST deviation typically < 2mV
            (abs(features[2]) < 2.0, "ST dev V5"),          
            (abs(features[3]) < 3.0, "T mag V2"),           # T wave typically < 3mV
            (abs(features[4]) < 3.0, "T mag V5"),
            (0.04 <= features[5] <= 0.25, "QRS duration"),  # QRS: 40-250ms
            (0.01 <= features[6] <= 100, "R/S ratio"),      # R/S ratio reasonable range
            (0.01 <= features[7] <= 5.0, "RMS")             # RMS should be positive and reasonable
        ]
        
        for is_valid, feature_name in checks:
            if not is_valid:
                if debug:
                    print(f"Invalid {feature_name}: {features}")
                return None
        
        return features
        
    except Exception as e:
        if debug:
            print(f"Extraction error: {e}")
        return None


In [5]:
# --- Feature Extraction Pipeline ---
def run_feature_extraction(df_path, save_dir, debug_mode=False):
    """
    Process all ECG signals and extract features.
    """
    df = pd.read_csv(df_path)
    os.makedirs(save_dir, exist_ok=True)
    
    processed_rows = []
    failed_count = 0
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting Features"):
        npy_path = row['processed_npy_path']
        original_base_path = row['hea_path']
        try:
            # Load and transpose signal to (12, 2500)
            signal = np.load(npy_path)
            
            # Handle both (2500, 12) and (12, 2500) formats
            if signal.shape == (SAMPLES, LEAD_COUNT):
                signal = signal.T
            elif signal.shape != (LEAD_COUNT, SAMPLES):
                if debug_mode:
                    print(f"Unexpected shape {signal.shape} for {npy_path}")
                failed_count += 1
                continue
            
            # Extract features
            features = extract_cad_features(signal, debug=debug_mode)
            
            if features is None:
                failed_count += 1
                if debug_mode and idx < 10:  # Only debug first 10 failures
                    print(f"Failed to extract features from: {os.path.basename(npy_path)}")
                continue
            
            # Save features
            # --- 3. CRITICAL SAVING LOGIC: Replicate Folder Structure ---
            
            # Extract the relative path structure from the original path
            # e.g., gets 'p1125/p11259533/s44630244/44630244'
            structure_path = original_base_path.split('files')[-1].lstrip(os.sep) 
            
            # Replace the extension to get the structure path for the .npy file
            relative_feature_path = structure_path.replace('.hea', '_features.npy')
            
            # Construct the full absolute path for saving the feature file
            feature_save_path = os.path.join(save_dir, relative_feature_path)
            
            # Ensure the deep target directory exists
            feature_dir = os.path.dirname(feature_save_path)
            os.makedirs(feature_dir, exist_ok=True)
            
            # Save the file (This must succeed now that the directory exists)
            np.save(feature_save_path, features)
            
            # Update metadata
            new_row = row.to_dict()
            new_row['feature_path'] = os.path.abspath(feature_save_path)
            processed_rows.append(new_row)
            
        except Exception as e:
            failed_count += 1
            if debug_mode:
                print(f"Error processing {os.path.basename(npy_path)}: {e}")
            continue
    
    return pd.DataFrame(processed_rows), failed_count

In [6]:
# # Add this before your main execution
# signal = np.load(df.iloc[0]['processed_npy_path'])
# print(f"Signal shape: {signal.shape}")
# print(f"Signal stats: min={signal.min():.3f}, max={signal.max():.3f}, mean={signal.mean():.3f}")

# features = extract_cad_features(signal.T if signal.shape == (2500, 12) else signal, debug=True)
# print(f"Features: {features}")

In [7]:
print("Starting clinical feature extraction...")
print(f"Processing {len(pd.read_csv(CSV_PATH))} records...")

# Set debug_mode=True to see detailed error messages for first few failures
final_metadata_df, failed = run_feature_extraction(
    CSV_PATH, 
    FEATURE_SAVE_DIR, 
    debug_mode=True
)

Starting clinical feature extraction...
Processing 31920 records...


Extracting Features:   0%|          | 54/31920 [00:15<2:14:26,  3.95it/s]

Too few R-peaks detected: 1


Extracting Features:   0%|          | 143/31920 [00:40<2:12:52,  3.99it/s]

Too few R-peaks detected: 1


...

Extracting Features: 100%|█████████▉| 31844/31920 [1:42:08<00:11,  6.42it/s]

Delineation failed: cannot convert float NaN to integer


Extracting Features: 100%|█████████▉| 31864/31920 [1:42:11<00:08,  7.00it/s]

Invalid R/S ratio: [ 0.0010093   0.00091406  0.         -0.00229481 -0.00093022  0.098
  0.00989033  0.13849907]


Extracting Features: 100%|██████████| 31920/31920 [1:42:20<00:00,  5.20it/s]


In [8]:
# Save the updated, clean metadata to disk
final_metadata_df.to_csv(OUTPUT_CSV_PATH, index=False)

In [9]:
initial_count = len(pd.read_csv(CSV_PATH))
final_count = len(final_metadata_df)

print(f"\n{'='*60}")
print(f"Extraction Complete!")
print(f"{'='*60}")
print(f"Initial Records:        {initial_count}")
print(f"Successfully Processed: {final_count} ({final_count/initial_count*100:.1f}%)")
print(f"Failed:                 {failed} ({failed/initial_count*100:.1f}%)")
print(f"{'='*60}")
print(f"Output saved to: '{OUTPUT_CSV_PATH}'")


Extraction Complete!
Initial Records:        31920
Successfully Processed: 31376 (98.3%)
Failed:                 544 (1.7%)
Output saved to: 'metadata_with_features.csv'


In [10]:
# Check to see how many records were filtered out gemini
initial_count = len(pd.read_csv(CSV_PATH))
final_count = len(final_metadata_df)


print(f"Extraction complete.")
print(f"Initial Records (Preprocessed): {initial_count}")
print(f"Records with Valid Features: {final_count}")
print(f"Filtered Records (Bad Quality): {initial_count - final_count}")
print(f"Clean metadata saved to '{OUTPUT_CSV_PATH}'")

Extraction complete.
Initial Records (Preprocessed): 31920
Records with Valid Features: 31376
Filtered Records (Bad Quality): 544
Clean metadata saved to 'metadata_with_features.csv'


In [ ]:
# import numpy as np
# import neurokit2 as nk
# import pandas as pd
# import os
# from tqdm import tqdm
# from scipy.signal import medfilt
# import warnings

# warnings.filterwarnings('ignore')

# # --- Configuration ---
# FS = 500
# LEAD_COUNT = 12
# SAMPLES = 2500

# # Standard 12-lead ECG ordering: I, II, III, aVR, aVL, aVF, V1, V2, V3, V4, V5, V6
# LEAD_II = 1   # For rhythm analysis
# LEAD_V1 = 6   # For R/S ratio
# LEAD_V2 = 7   # For ST/T analysis
# LEAD_V5 = 10  # For ST/T analysis and baseline

# CSV_PATH = 'baseline_filtered_ecg_patients_2.csv'
# FEATURE_SAVE_DIR = './extracted_clinical_features'
# OUTPUT_CSV_PATH = 'metadata_with_features.csv'

# # --- Feature Extraction Function ---
# def extract_cad_features(signal, fs=FS, debug=False):
#     """
#     Extracts 8 clinically relevant features for CAD severity prediction.
    
#     Returns:
#         np.ndarray of 8 features or None if extraction fails
#     """
#     if signal.shape[0] != LEAD_COUNT or signal.shape[1] != SAMPLES:
#         if debug:
#             print(f"Invalid shape: {signal.shape}")
#         return None
    
#     # Use Lead II for rhythm analysis
#     lead_ii = np.ascontiguousarray(signal[LEAD_II, :], dtype=np.float64)
    
#     features = np.zeros(8)
    
#     try:
#         # Step 1: R-Peak Detection
#         r_peaks_info = nk.ecg_findpeaks(lead_ii, sampling_rate=fs, method='neurokit')
#         r_peaks = np.array(r_peaks_info['ECG_R_Peaks'])
        
#         # Require at least 3 R-peaks for reliable feature extraction
#         if len(r_peaks) < 3:
#             if debug:
#                 print(f"Too few R-peaks detected: {len(r_peaks)}")
#             return None
        
#         # Step 2: Delineation with better error handling
#         try:
#             _, waves_info = nk.ecg_delineate(
#                 lead_ii, 
#                 rpeaks=r_peaks, 
#                 sampling_rate=fs, 
#                 method="dwt"
#             )
#         except Exception as e:
#             if debug:
#                 print(f"Delineation failed: {e}")
#             return None
        
#         # Step 3: Find the central beat
#         center_idx = np.argmin(np.abs(r_peaks - SAMPLES // 2))
#         r_peak_idx = r_peaks[center_idx]
        
#         # Step 4: Extract wave boundaries with validation
#         def get_wave_index(wave_dict, key, beat_idx):
#             """Safely extract wave index with None checking"""
#             if key not in wave_dict:
#                 return None
#             val = wave_dict[key][beat_idx]
#             if val is None or np.isnan(val):
#                 return None
#             return int(val)
        
#         q_onset = get_wave_index(waves_info, 'ECG_Q_Peaks', center_idx)
#         s_offset = get_wave_index(waves_info, 'ECG_S_Peaks', center_idx)
#         t_peak = get_wave_index(waves_info, 'ECG_T_Peaks', center_idx)
        
#         # If any critical point is missing, try alternative detection
#         if q_onset is None:
#             # Q onset approximation: 40ms before R peak
#             q_onset = max(0, r_peak_idx - int(0.04 * fs))
        
#         if s_offset is None:
#             # S offset approximation: 40ms after R peak
#             s_offset = min(SAMPLES - 1, r_peak_idx + int(0.04 * fs))
        
#         if t_peak is None:
#             # T peak approximation: 200ms after R peak
#             t_peak_approx = min(SAMPLES - 1, r_peak_idx + int(0.2 * fs))
#             # Find actual peak in window
#             search_start = r_peak_idx + int(0.1 * fs)
#             search_end = min(SAMPLES, r_peak_idx + int(0.35 * fs))
#             if search_end > search_start:
#                 t_peak = search_start + np.argmax(lead_ii[search_start:search_end])
#             else:
#                 t_peak = t_peak_approx
        
#         # Validate indices
#         if not (0 <= q_onset < r_peak_idx < s_offset < SAMPLES):
#             if debug:
#                 print(f"Invalid wave sequence: Q={q_onset}, R={r_peak_idx}, S={s_offset}")
#             return None
        
#         if not (0 <= t_peak < SAMPLES):
#             if debug:
#                 print(f"Invalid T peak: {t_peak}")
#             return None
        
#         # Step 5: Calculate baselines for each lead independently
#         baseline_v2 = medfilt(signal[LEAD_V2, :], kernel_size=11)
#         baseline_v5 = medfilt(signal[LEAD_V5, :], kernel_size=11)
#         baseline_ii = medfilt(signal[LEAD_II, :], kernel_size=11)
        
#         # Step 6: Extract Features with robust error handling
        
#         # Feature 0: Q Wave Depth (Lead II)
#         q_region_start = max(0, q_onset)
#         q_region_end = min(SAMPLES, r_peak_idx)
#         if q_region_end > q_region_start:
#             q_peak_idx = q_region_start + np.argmin(signal[LEAD_II, q_region_start:q_region_end])
#             q_depth = baseline_ii[q_peak_idx] - signal[LEAD_II, q_peak_idx]
#             features[0] = q_depth if not (np.isnan(q_depth) or np.isinf(q_depth)) else 0
#         else:
#             features[0] = 0
        
#         # ST segment point: 60ms after S offset
#         st_point_idx = min(SAMPLES - 1, s_offset + int(0.06 * fs))
        
#         # Feature 1: ST Deviation in V2
#         st_dev_v2 = signal[LEAD_V2, st_point_idx] - baseline_v2[st_point_idx]
#         features[1] = st_dev_v2 if not (np.isnan(st_dev_v2) or np.isinf(st_dev_v2)) else 0
        
#         # Feature 2: ST Deviation in V5
#         st_dev_v5 = signal[LEAD_V5, st_point_idx] - baseline_v5[st_point_idx]
#         features[2] = st_dev_v5 if not (np.isnan(st_dev_v5) or np.isinf(st_dev_v5)) else 0
        
#         # Feature 3: T Wave Magnitude in V2
#         t_mag_v2 = signal[LEAD_V2, t_peak] - baseline_v2[t_peak]
#         features[3] = t_mag_v2 if not (np.isnan(t_mag_v2) or np.isinf(t_mag_v2)) else 0
        
#         # Feature 4: T Wave Magnitude in V5
#         t_mag_v5 = signal[LEAD_V5, t_peak] - baseline_v5[t_peak]
#         features[4] = t_mag_v5 if not (np.isnan(t_mag_v5) or np.isinf(t_mag_v5)) else 0
        
#         # Feature 5: QRS Duration (seconds)
#         qrs_duration = (s_offset - q_onset) / fs
#         features[5] = qrs_duration if 0.04 <= qrs_duration <= 0.20 else 0.10  # Physiologically valid range
        
#         # Feature 6: R/S Ratio in V1
#         qrs_region_v1 = signal[LEAD_V1, q_onset:s_offset]
#         if len(qrs_region_v1) > 0:
#             r_mag_v1 = np.max(qrs_region_v1) - np.median(qrs_region_v1)  # Relative to baseline
#             s_mag_v1 = np.median(qrs_region_v1) - np.min(qrs_region_v1)  # Relative to baseline
            
#             # Both should be positive; if not, use absolute values
#             r_mag_v1 = abs(r_mag_v1)
#             s_mag_v1 = abs(s_mag_v1)
            
#             if s_mag_v1 > 0.01:  # Threshold to avoid division by near-zero
#                 rs_ratio = r_mag_v1 / s_mag_v1
#                 features[6] = rs_ratio if not (np.isnan(rs_ratio) or np.isinf(rs_ratio)) else 1.0
#             else:
#                 features[6] = 1.0  # Default when S wave is absent/minimal
#         else:
#             features[6] = 1.0
        
#         # Feature 7: Lead II RMS
#         rms_ii = np.sqrt(np.mean(lead_ii**2))
#         features[7] = rms_ii if not (np.isnan(rms_ii) or np.isinf(rms_ii)) else 0
        
#         # Sanity check: all features should be finite and within reasonable ranges
#         if np.any(~np.isfinite(features)):
#             if debug:
#                 print(f"Non-finite feature values detected: {features}")
#             return None
        
#         # Validate physiological ranges (relaxed for pathological cases)
#         checks = [
#             (abs(features[0]) < 2.0, "Q wave depth"),      # Q depth typically < 2mV
#             (abs(features[1]) < 2.0, "ST dev V2"),          # ST deviation typically < 2mV
#             (abs(features[2]) < 2.0, "ST dev V5"),          
#             (abs(features[3]) < 3.0, "T mag V2"),           # T wave typically < 3mV
#             (abs(features[4]) < 3.0, "T mag V5"),
#             (0.04 <= features[5] <= 0.25, "QRS duration"),  # QRS: 40-250ms
#             (0.01 <= features[6] <= 100, "R/S ratio"),      # R/S ratio reasonable range
#             (0.01 <= features[7] <= 5.0, "RMS")             # RMS should be positive and reasonable
#         ]
        
#         for is_valid, feature_name in checks:
#             if not is_valid:
#                 if debug:
#                     print(f"Invalid {feature_name}: {features}")
#                 return None
        
#         return features
        
#     except Exception as e:
#         if debug:
#             print(f"Extraction error: {e}")
#         return None


# # --- Feature Extraction Pipeline ---
# def run_feature_extraction(df_path, save_dir, debug_mode=False):
#     """
#     Process all ECG signals and extract features.
#     """
#     df = pd.read_csv(df_path)
#     os.makedirs(save_dir, exist_ok=True)
    
#     processed_rows = []
#     failed_count = 0
    
#     for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting Features"):
#         npy_path = row['processed_npy_path']
#         original_base_path = row['hea_path']
#         try:
#             # Load and transpose signal to (12, 2500)
#             signal = np.load(npy_path)
            
#             # Handle both (2500, 12) and (12, 2500) formats
#             if signal.shape == (SAMPLES, LEAD_COUNT):
#                 signal = signal.T
#             elif signal.shape != (LEAD_COUNT, SAMPLES):
#                 if debug_mode:
#                     print(f"Unexpected shape {signal.shape} for {npy_path}")
#                 failed_count += 1
#                 continue
            
#             # Extract features
#             features = extract_cad_features(signal, debug=debug_mode)
            
#             if features is None:
#                 failed_count += 1
#                 if debug_mode and idx < 10:  # Only debug first 10 failures
#                     print(f"Failed to extract features from: {os.path.basename(npy_path)}")
#                 continue
            
#             # Save features
#             # --- 3. CRITICAL SAVING LOGIC: Replicate Folder Structure ---
            
#             # Extract the relative path structure from the original path
#             # e.g., gets 'p1125/p11259533/s44630244/44630244'
#             structure_path = original_base_path.split('files')[-1].lstrip(os.sep) 
            
#             # Replace the extension to get the structure path for the .npy file
#             relative_feature_path = structure_path.replace('.hea', '_features.npy')
            
#             # Construct the full absolute path for saving the feature file
#             feature_save_path = os.path.join(save_dir, relative_feature_path)
            
#             # Ensure the deep target directory exists
#             feature_dir = os.path.dirname(feature_save_path)
#             os.makedirs(feature_dir, exist_ok=True)
            
#             # Save the file (This must succeed now that the directory exists)
#             np.save(feature_save_path, features)
            
#             # Update metadata
#             new_row = row.to_dict()
#             new_row['feature_path'] = os.path.abspath(feature_save_path)
#             processed_rows.append(new_row)
            
#         except Exception as e:
#             failed_count += 1
#             if debug_mode:
#                 print(f"Error processing {os.path.basename(npy_path)}: {e}")
#             continue
    
#     return pd.DataFrame(processed_rows), failed_count
	
# print("Starting clinical feature extraction...")
# print(f"Processing {len(pd.read_csv(CSV_PATH))} records...")

# # Set debug_mode=True to see detailed error messages for first few failures
# final_metadata_df, failed = run_feature_extraction(
#     CSV_PATH, 
#     FEATURE_SAVE_DIR, 
#     debug_mode=True
# )

# # Save the updated, clean metadata to disk
# final_metadata_df.to_csv(OUTPUT_CSV_PATH, index=False)

# initial_count = len(pd.read_csv(CSV_PATH))
# final_count = len(final_metadata_df)

# print(f"\n{'='*60}")
# print(f"Extraction Complete!")
# print(f"{'='*60}")
# print(f"Initial Records:        {initial_count}")
# print(f"Successfully Processed: {final_count} ({final_count/initial_count*100:.1f}%)")
# print(f"Failed:                 {failed} ({failed/initial_count*100:.1f}%)")
# print(f"{'='*60}")
# print(f"Output saved to: '{OUTPUT_CSV_PATH}'")

# # Check to see how many records were filtered out gemini
# initial_count = len(pd.read_csv(CSV_PATH))
# final_count = len(final_metadata_df)


# print(f"Extraction complete.")
# print(f"Initial Records (Preprocessed): {initial_count}")
# print(f"Records with Valid Features: {final_count}")
# print(f"Filtered Records (Bad Quality): {initial_count - final_count}")
# print(f"Clean metadata saved to '{OUTPUT_CSV_PATH}'")

